<h1>Coding a K-Nearest Neighbors Classifier from scratch.</h1>

This notebook is an introductory exercise of the K-Nearest Neighbors (KNN) method for classification.

<h2>Introduction</h2>
The K-Nearest Neighbors algorithm is a supervised learning algorithm for classifying data. The way it works is by treating the features of the items as dimensions in an Euclidean space so you can see each item as a point in the space. Its premise is that items from a same category should have similar values in their features, therefore be close to each other in the space.

In order to classify an unknown item the KNN algorithm calculates its distance in the space to every other known item and choses the nearest  as reference to the prediction. How many will it chose? K. That's why it is called k-nearest neighbors. After knowing the categories from its k-nearest neighbors the algorithm will take a votation and select the most common category as the predicted category.

<h1>Implementation</h1>
Let's write a general implementation and then try it out with a dataset.
<h2>Import libraries</h2>
Start by importing some useful libraries:
* ***Math*** for calculating the square root of a number
* ***Pandas*** for dealing with the data
* ***Counter*** for selecting the most common category
* ***Matplotlib*** for ploting some results

In [1]:
import math
import pandas as pd
from collections import Counter

<h2>Calculate the distances</h2>
In order to know which are the nearest points in the space you are going to need a way to calculate the distances in the space. There are different ways to define and calculate the required distance, I'm going to choose the Euclidean distance as it is more intuitive to me from a geometry point of view.

In [2]:
def euclidean_dist(xi,xj):
    """
    Calculates the euclidean distance in a N-dimensional space between the points xi and xj
    
    Parameters:
        xi, xj: numeric list or tuple representing each point.
        
    Returns:
        distance (float): the euclidean distance between the points.
    """
    
    distance_squared = 0
    for d in range(len(xi)):
        distance_squared += (xj[d] - xi[d]) ** 2
    return math.sqrt(distance_squared)

In [3]:
x1 = (2,1,1)
x2 = (1,0,0)
euclidean_dist(x1,x2)

1.7320508075688772

Now that we have defined a distance we can calculate that distance from a given point, which label is unknown, to all the other points which category labels we already know.
So let's suppose we have the data in the form of a Pandas DataFrame and we know wich columns we want to take as dimensions for the Euclidean space in wich we are going to calculate distance. Then we will need to subset the dataframe by those columns and convert each row into the appropiate type in order to apply our distance function.

In [4]:
def calc_all_distances(xi, data, columns):
    """
    Calculates the Euclidean distance between a point and all the points in the data.
    
    Parameters:
        xi      : Numeric list or tuple representing the point.
        data    : Pandas dataframe with the training data.
                  Each row will represent a point in the N-dimensional space.
        columns : A list of the columns name to take as the dimensions of the Euclidean space. 
        
    Returns:
        distances (Series): the euclidean distances between the points.
        
    """
    #Convert data frame to points in n-dimensional space
    points = data[columns].apply(tuple,axis=1)
    #Calculate distance for each row
    distances = points.apply(lambda x: euclidean_dist(xi,x))
    return distances

The above function calculates all the distances to a point by:
* Subseting the data by the desired columns and formating the rows into tuples (points).
* Applying the *euclidean_dist* function to each row

<h2>Prediction</h2>
For making a classification you have to calculate all the distances from a point of interest to the other known points, and then select the k-nearest to that point to let them vote. The category with the most votes is going to be the prediction.
To achieve that we are going to sort the result of *calc_all_distances* and take the indices of the k-smallest distances. With the indices we can retrieve their labels in the target column and count them to choose the most common.

In [5]:
def predict_knn(xi, data, columns, target_column, k):
    """
    Predicts the label for a given point by taking votes from its k-nearest neigbors
    
    Parameters:
        xi           : Numeric list or tuple representing the point.
        data         : Pandas dataframe with the training data.
                       Each row will represent a point in the N-dimensional space.
        columns      : A list of the columns names to take as the dimensions of the Euclidean space. 
        target_column: The name of the target feature
        k            : The number of nearest neighbors to take into account
        
    Returns:
        prediction (str): the predicted label for the point
    
    """
    
    all_distances = calc_all_distances(xi,data,columns)
    k_nearest = list(all_distances.sort_values()[:k].index)
    k_nearest_labels = data.loc[k_nearest, target_column]
    poll = Counter(k_nearest_labels)
    return poll.most_common()[0][0]

That's it! Now let's test it with data.
<h1>Using the implementation</h1>
I'm going to use the IRIS data set as its length and features makes it appropiate to use with KNN classification. You cand read more about the data in the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Iris)
<h2>Load the data</h2>

In [6]:
iris_data = pd.read_csv('/kaggle/input/iris/Iris.csv')
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


For using our *predict_knn* function we need to specify the columns that will form the dimensions of the Euclidean space and the target column.

In [7]:
feature_columns = list(iris_data.columns[1:5])
print(feature_columns)
target_column = iris_data.columns[5]
print(target_column)

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
Species


Now let's create a ficticious flower point and see how it gets classified. I'm gonna arbitrarily choose k=3. 

In [8]:
flower_point = (5.0, 4.0, 2.0, 1.5)
k = 3
print(predict_knn(flower_point,
                  iris_data, 
                  feature_columns,
                  target_column,
                  k))

Iris-setosa


Right now it seems to be working just fine. But what about the accuracy of our model?
Because we don't have any other data to test let's check its perfomance with the train data. We are going to try to predict each point category in the dataset, by using the rest of the dataset each time, and compare the prediction with the actual category label.

In [9]:
predictions = list()
for row in iris_data.itertuples():
    dict_row = row._asdict()
    xi = list()
    for col in feature_columns:
        xi.append(dict_row[col])
    predictions.append(predict_knn(xi,
                                   iris_data.drop(index=row.Index), 
                                   feature_columns, 
                                   target_column, 
                                   k))
        
print(predictions)

['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'I

The code above predicts the category for each row (you can expand the output to check the results). Now lets check how well it classified the data.

In [10]:
correct_predictions = iris_data.loc[:,target_column] == predictions
accuracy = sum(correct_predictions)/len(predictions)
print(accuracy)

0.96


As you can see we got a pretty good accuracy score. The reason why it does not have a perfect accuracy score is because for these featuras the point clouds of the categories Iris-versicolor and Iris-virginica are close to each other and somewhat overlapping in the Euclidean space. Therefore the points inside that overlapping zone are going to be hard to classify properly.

You could try different k values in order to try to improve that result, but you have to be carefull to not overfit the model to this train data.